# Merging all data sets

#### this notebook merges all wrangled data sets: orders, prior, products and customers.
#### the departments data are not essential, they will be used as a data dictionary in the analysis.

## Importing libraries and data

In [1]:
#importing the libraries

import pandas as pd
import numpy as np
import os

In [2]:
#importing datasets:
path = r'C:\Users\chris\Documents\Instacart Basket Analysis'
ords = pd.read_pickle(os.path.join(path,'02 Data', 'Prepared Data', 'orders_wrangled.pkl'))
prods = pd.read_pickle(os.path.join(path,'02 Data', 'Prepared Data', 'products_wrangled.pkl'))
prior = pd.read_pickle(os.path.join(path,'02 Data', 'Prepared Data', 'prior_wrangled.pkl'))
cust = pd.read_pickle(os.path.join(path,'02 Data', 'Prepared Data', 'customers_wrangled.pkl'))

### Planning the merges

In [3]:
#finding KEYs for merge: ords
ords.columns

Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'order_hour_of_day', 'days_since_last_order'],
      dtype='object')

In [4]:
#finding KEYs for merge: prods
prods.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices'], dtype='object')

In [5]:
#finding KEYs for merge: prior
prior.columns

Index(['order_id', 'product_id', 'add_to_cart_order', 'reordered'], dtype='object')

In [6]:
#finding KEYs for merge: cust
cust.columns

Index(['user_id', 'Gender', 'STATE', 'Age', 'date_joined', 'n_dependants',
       'fam_status', 'income'],
      dtype='object')

##### First merge has to be ords with prior (Key = 'order_id') > creating the matching Key with the prods (Key = 'product_id'). Finally we will add the customers (Key = 'user_id').

### 1) Merging orders with prior >> new dataframe: orders_products_combined

#### 1.a) Performing the standard merge, which is an INNER JOIN:

In [7]:
#pre-merge: shape of ords:
ords.shape

(3421083, 6)

In [8]:
#pre-merge: shape of prior:
prior.shape

(32434489, 4)

In [9]:
#merging on: order_id, INNER JOIN
orders_products_combined = ords.merge(prior, on = 'order_id', indicator = True)

In [10]:
#calling the merged dataframe
orders_products_combined

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196,1,0,both
1,2539329,1,1,2,8,NaN,14084,2,0,both
2,2539329,1,1,2,8,NaN,12427,3,0,both
3,2539329,1,1,2,8,NaN,26088,4,0,both
4,2539329,1,1,2,8,NaN,26405,5,0,both
...,...,...,...,...,...,...,...,...,...,...
32434484,2977660,206209,13,1,12,7.0,14197,5,1,both
32434485,2977660,206209,13,1,12,7.0,38730,6,0,both
32434486,2977660,206209,13,1,12,7.0,31477,7,0,both
32434487,2977660,206209,13,1,12,7.0,6567,8,0,both


In [11]:
# after merge check: size of merged dataframe:
orders_products_combined.shape

(32434489, 10)

In [12]:
# checking the merge flag:
orders_products_combined['_merge'].value_counts()

both          32434489
left_only            0
right_only           0
Name: _merge, dtype: int64

#### 1.b) Checking results with the contrary merge, performing an OUTER JOIN:

In [13]:
#merging on: order_id, OUTER JOIN
orders_products_combined_outer = ords.merge(prior, on = 'order_id', indicator = True, how = 'outer')

In [14]:
# checking the merge flag:
orders_products_combined_outer['_merge'].value_counts()

both          32434489
left_only       206209
right_only           0
Name: _merge, dtype: int64

##### The OUTER JOIN shows, that there are more than 200k rows without a match. Reasons for this need to be checked in the next step.

In [15]:
# filter all not merged rows:
unmerged_data = orders_products_combined_outer[orders_products_combined_outer['_merge'] != 'both']

In [16]:
# calling the unmerged_data:
unmerged_data

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge
59,1187899,1,11,4,8,14.0,NaN,NaN,NaN,left_only
255,1492625,2,15,1,11,30.0,NaN,NaN,NaN,left_only
344,2774568,3,13,5,15,11.0,NaN,NaN,NaN,left_only
363,329954,4,6,3,12,30.0,NaN,NaN,NaN,left_only
401,2196797,5,5,0,11,6.0,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...
32639379,1716008,206205,4,1,16,10.0,NaN,NaN,NaN,left_only
32639665,1043943,206206,68,0,20,0.0,NaN,NaN,NaN,left_only
32639889,2821651,206207,17,2,13,14.0,NaN,NaN,NaN,left_only
32640567,803273,206208,50,5,11,4.0,NaN,NaN,NaN,left_only


In [17]:
# apparently th product_id is missing. Subsetting to be sure:
subset_by_product_id_outer = unmerged_data.loc[:, 'product_id']

In [18]:
# if this is the reason, the number of rows equals the number of unmerged data:
subset_by_product_id_outer.shape

(206209,)

In [19]:
# comparative subset in the inner join merge:
subset_by_product_id_inner = orders_products_combined.loc[orders_products_combined['product_id'].isna(), 'product_id']

In [20]:
subset_by_product_id_inner.shape

(0,)

##### In the orders-dataframe were over 200k rows without a match in the prior_orders-dataframe. In the unmerging rows, essential data is missing, above all the product_id, which will be the Key for the merge with the products dataframe. Therefore, we will continue with the inner joined dataframe.

### 2) Merge 'orders_products_combined' with 'prods' >> new dataframe: orders_products_merged

#### 2.a) Performing the standard merge, which is an INNER JOIN:

In [21]:
#pre-merge: shape of orders_products_combined:
orders_products_combined.shape

(32434489, 10)

In [22]:
#pre-merge: shape of prods:
prods.shape

(49670, 5)

In [23]:
#merging on: product_id, INNER JOIN
orders_products_merged = orders_products_combined.merge(prods, on = 'product_id')

In [24]:
#calling the merged df:
orders_products_merged

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices
0,2539329,1,1,2,8,NaN,196,1,0,both,Soda,77,7,9.0
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32399727,1320836,202557,17,2,15,1.0,43553,2,1,both,Orange Energy Shots,64,7,3.7
32399728,31526,202557,18,5,11,3.0,43553,2,1,both,Orange Energy Shots,64,7,3.7
32399729,758936,203436,1,2,7,NaN,42338,4,0,both,"Zucchini Chips, Pesto",50,19,6.9
32399730,2745165,203436,2,3,5,15.0,42338,16,1,both,"Zucchini Chips, Pesto",50,19,6.9


In [25]:
# after merge check: size of merged dataframe:
orders_products_merged.shape

(32399732, 14)

In [26]:
# checking the merge flag:
orders_products_merged['_merge'].value_counts()

both          32399732
left_only            0
right_only           0
Name: _merge, dtype: int64

#### 2.b) Checking results with the contrary merge, performing an OUTER JOIN:

In [27]:
#merging on: product_id, OUTER JOIN
orders_products_merged_outer = orders_products_combined.merge(prods, on = 'product_id', how = 'outer')

In [28]:
# checking the merge flag:
orders_products_merged_outer['_merge'].value_counts()

both          32435059
left_only            0
right_only           0
Name: _merge, dtype: int64

##### No unmerged rows here. No further investigations required.

### 3) merging 'orders_products_merged' with 'cust' >> new dataframe: orders_products_all_merge

#### 3.a) Performing the standard merge, which is an INNER JOIN:

In [29]:
#pre-merge: shape of orders_products_merged:
orders_products_merged.shape

(32399732, 14)

In [30]:
#pre-merge: shape of cust
cust.shape

(206209, 8)

In [31]:
#merging on: user_id, INNER JOIN
orders_products_all_merge = orders_products_merged.merge(cust, on = 'user_id')

In [32]:
# after merge check: size of merged dataframe:
orders_products_all_merge.shape

(32399732, 21)

In [33]:
orders_products_all_merge

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,_merge,...,aisle_id,department_id,prices,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,NaN,196,1,0,both,...,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,both,...,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,both,...,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,both,...,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,both,...,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32399727,156685,106143,26,4,23,5.0,19675,1,1,both,...,94,7,10.7,Male,Hawaii,25,2017-05-26,0,single,53755
32399728,484769,66343,1,6,11,NaN,47210,1,0,both,...,15,12,8.1,Female,Tennessee,22,2017-09-12,3,married,46151
32399729,1561557,66343,2,1,11,30.0,47210,1,1,both,...,15,12,8.1,Female,Tennessee,22,2017-09-12,3,married,46151
32399730,276317,66343,3,6,15,19.0,47210,1,1,both,...,15,12,8.1,Female,Tennessee,22,2017-09-12,3,married,46151


In [34]:
# checking the merge flag:
orders_products_all_merge['_merge'].value_counts()

both          32399732
left_only            0
right_only           0
Name: _merge, dtype: int64

##### 3.b) Checking results with the contrary merge, performing an OUTER JOIN:

In [35]:
#merging on: product_id, OUTER JOIN
orders_products_all_merge_outer = orders_products_merged.merge(cust, on = 'user_id', how = 'outer')

In [36]:
# checking the merge flag:
orders_products_all_merge_outer['_merge'].value_counts()

both          32399732
left_only            0
right_only           0
Name: _merge, dtype: int64

##### No unmerged rows here. No further investigations required.¶

### 4) check: looking for duplicates after merging

In [41]:
# looking for duplicates:
df_dups = orders_products_all[orders_products_all.duplicated()]
df_dups

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,Gender,STATE,Age,date_joined,n_dependants,fam_status,income


##### No duplicates.

### 5) drop the '_merge' column >>> FINAL dataframe: orders_products_all.pkl

In [37]:
# dropping the automatically generated merge-column. We don't need it in the final dataframe.
orders_products_all = orders_products_all_merge.drop('_merge', axis=1)

In [38]:
orders_products_all

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32399727,156685,106143,26,4,23,5.0,19675,1,1,Organic Raspberry Black Tea,94,7,10.7,Male,Hawaii,25,2017-05-26,0,single,53755
32399728,484769,66343,1,6,11,NaN,47210,1,0,Fresh Farmed Tilapia Fillet,15,12,8.1,Female,Tennessee,22,2017-09-12,3,married,46151
32399729,1561557,66343,2,1,11,30.0,47210,1,1,Fresh Farmed Tilapia Fillet,15,12,8.1,Female,Tennessee,22,2017-09-12,3,married,46151
32399730,276317,66343,3,6,15,19.0,47210,1,1,Fresh Farmed Tilapia Fillet,15,12,8.1,Female,Tennessee,22,2017-09-12,3,married,46151


In [39]:
orders_products_all.shape

(32399732, 20)

## exporting the final dataframe orders_products_all.pkl

In [40]:
# exporting the merged dataframe as a pkl:
orders_products_all.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_all.pkl'))